# Linjär Regressionsanalys av Bostadspriser i Kalifornien

## Introduktion

I denna laboration undersöks sambandet mellan ett antal variabler som beskriver bostadsområden i Kalifornien och det medianbostadsvärde som observerats i respektive område. Datasetet, som härrör från folkräkningen 1990, innehåller 20 640 observationer och beskriver varje censusblock med uppgifter om bland annat medianinkomst, husålder, antal rum, befolkningsstorlek, geografisk position och närheten till havet.

Responsvariabeln i analysen är `median_house_value`, det vill säga medianpriset på bostäder i ett givet block. Syftet är att med hjälp av multipel linjär regression (OLS) skatta hur de tillgängliga variablerna bidrar till att förklara prisvariationen, och att undersöka modellens styrkor och svagheter med statistiska verktyg: signifikanstest, konfidensintervall, Pearson-korrelation och residualdiagnostik.

## Metod

Modellen implementeras i en egen klass (`LinearRegression` i `linear_regression.py`) som enbart använder numpy för linjär algebra och scipy.stats för fördelningar och test. Inga färdiga regressionsimplementationer eller visualiseringsbibliotek används.

Arbetsgången börjar med att läsa in CSV-filen och konvertera alla värden till numeriska arrayer. Kolumnen `total_bedrooms` innehåller 207 saknade värden, vilka imputeras med kolumnens medelvärde för att bevara stickprovsstorleken. Eftersom datasetet innehåller flera censusblock med identiska koordinater aggregeras sedan observationerna per unik (longitude, latitude)-kombination: additiva variabler (`total_rooms`, `total_bedrooms`, `population`, `households`) summeras, medan nivåvariabler (`housing_median_age`, `median_income`, `median_house_value`) ersätts av medelvärdet inom gruppen. Dessutom tas alla observationer med `median_house_value` = 500 001 bort, eftersom detta värde utgör en artificiell cap som bryter mot OLS-antagandena.

Den kategoriska variabeln `ocean_proximity` one-hot-kodas med referenskategorin `<1H OCEAN` utelämnad (drop-first) för att undvika linjärt beroende med interceptet.

Analysen genomförs i tre steg. Först skattas en fullständig modell med samtliga originalvariabler för att identifiera multikollinearitet och andra problem. Därefter konstrueras en förbättrad modell där de starkt korrelerade blockstorlek-variablerna ersätts med kvotvariabler (`rooms_per_household`, `people_per_household`) och koordinaterna ersätts av ett enda avståndsmått (`distance_to_centroid`). Slutligen angrips det kvarvarande problemet — heteroskedasticitet och icke-normalfördelade residualer — genom en log-linjär modell: responsvariabeln log-transformeras och de numeriska särdragen z-standardiseras. Log-transformationen bygger på insikten att bostadspriser i grunden är multiplikativa — en procentuell felterm är mer realistisk än en absolut — och den gör OLS-antagandena mer rimliga.

Koefficienterna skattas med normalequationen $\hat{\beta} = (X^TX)^{-1}X^Ty$. Implementationen använder `np.linalg.pinv` i stället för en vanlig invers, vilket innebär att beräkningen inte kraschar om $X^TX$ är singulär eller nära-singulär. Det är dock viktigt att förstå att pseudoinversen inte löser de statistiska problem som multikollinearitet medför — den ger enbart en numerisk lösning.

Efter skattningen utvärderas varje modell genom F-test och R² för övergripande signifikans, t-test och konfidensintervall för enskilda koefficienter, Pearson-korrelation för att identifiera multikollinearitet, samt residualstatistik för att bedöma modellens antaganden.

In [23]:
import numpy as np
import csv
from scipy.stats import normaltest
from linear_regression import LinearRegression

## 1. Datainläsning och förbehandling

Datasetet läses in från `housing.csv` med Pythons csv-modul. Varje rad representerar ett censusblock. De åtta numeriska kolumnerna konverteras till flyttal, och tomma celler tolkas som saknade värden (NaN). Dessa ersätts med respektive kolumns medelvärde.

Eftersom datasetet innehåller flera block med identiska koordinater (samma longitud och latitud) aggregeras sedan observationerna per unik geografisk punkt: additiva variabler summeras och nivåvariabler medelvärderas. Därefter tas alla observationer bort där `median_house_value` är exakt 500 001, eftersom detta värde representerar en artificiell övre gräns i datasetet. Att behålla dessa censurerade observationer skulle systematiskt snedvrida modellens residualer i det övre prisintervallet.

In [ ]:
path = 'housing.csv'
with open(path, newline='', encoding='utf-8') as f:
    r = csv.DictReader(f)
    rows = list(r)
cols_num = [
    'longitude','latitude','housing_median_age','total_rooms','total_bedrooms',
    'population','households','median_income'
]
y_col = 'median_house_value'
cat_col = 'ocean_proximity'

n_raw = len(rows)
X_raw = np.empty((n_raw, len(cols_num)), dtype=float)
y_raw = np.empty(n_raw, dtype=float)
cat_raw = np.empty(n_raw, dtype=object)
for i, row in enumerate(rows):
    for j, c in enumerate(cols_num):
        v = row[c]
        X_raw[i, j] = float(v) if v != '' else np.nan
    y_raw[i] = float(row[y_col])
    cat_raw[i] = row[cat_col]

missing_before = {c: int(np.isnan(X_raw[:, j]).sum()) for j, c in enumerate(cols_num)}

col_means = np.nanmean(X_raw, axis=0)
nan_inds = np.where(np.isnan(X_raw))
X_raw[nan_inds] = np.take(col_means, nan_inds[1])

sum_cols = {'total_rooms', 'total_bedrooms', 'population', 'households'}
mean_cols = {'housing_median_age', 'median_income'}
col_idx = {c: j for j, c in enumerate(cols_num)}

groups = {}
for i in range(n_raw):
    key = (X_raw[i, col_idx['longitude']], X_raw[i, col_idx['latitude']])
    if key not in groups:
        groups[key] = []
    groups[key].append(i)

n_unique = len(groups)
X_agg = np.empty((n_unique, len(cols_num)), dtype=float)
y_agg = np.empty(n_unique, dtype=float)
cat_agg = np.empty(n_unique, dtype=object)

for g_idx, (key, indices) in enumerate(groups.items()):
    idx_arr = np.array(indices)
    for j, c in enumerate(cols_num):
        if c in sum_cols:
            X_agg[g_idx, j] = np.sum(X_raw[idx_arr, j])
        elif c in mean_cols:
            X_agg[g_idx, j] = np.mean(X_raw[idx_arr, j])
        else:
            X_agg[g_idx, j] = X_raw[idx_arr[0], j]
    y_agg[g_idx] = np.mean(y_raw[idx_arr])
    cat_agg[g_idx] = cat_raw[indices[0]]

cap_mask = y_agg < 500001
X_num = X_agg[cap_mask]
y = y_agg[cap_mask]
cat = cat_agg[cap_mask]
n = len(y)

{
    'observationer_ratt': n_raw,
    'saknade_varden_fore_imputering': {k: v for k, v in missing_before.items() if v > 0},
    'unika_koordinater': n_unique,
    'borttagna_cappade (y=500001)': int((~cap_mask).sum()),
    'observationer_efter_rensning': n,
}

{'observationer': 20640,
 'numeriska_sardrag': 8,
 'kategorisk_variabel': 'ocean_proximity',
 'responsvariabel': 'median_house_value',
 'saknade_varden_fore_imputering': {'total_bedrooms': 207},
 'imputering': 'kolumnmedelvarde'}

## 2. Explorativ dataanalys (EDA)

För att få en överblick av datasetet beräknas deskriptiv statistik (minimum, medelvärde, median, maximum och standardavvikelse) för samtliga numeriska variabler samt responsvariabeln. Dessutom undersöks fördelningen av den kategoriska variabeln `ocean_proximity`.

In [ ]:
all_cols = cols_num + [y_col]
all_data = np.column_stack([X_num, y])

descriptive = {}
for j, name in enumerate(all_cols):
    col = all_data[:, j]
    descriptive[name] = {
        'min': round(float(np.min(col)), 2),
        'medel': round(float(np.mean(col)), 2),
        'median': round(float(np.median(col)), 2),
        'max': round(float(np.max(col)), 2),
        'std': round(float(np.std(col)), 2),
    }

ocean_categories, ocean_counts = np.unique(cat.astype(str), return_counts=True)
kategori_fordelning = {c: int(cnt) for c, cnt in zip(ocean_categories, ocean_counts)}

{
    'deskriptiv_statistik': descriptive,
    'ocean_proximity_fordelning': kategori_fordelning,
}

{'deskriptiv_statistik': {'longitude': {'min': -124.35,
   'medel': -119.57,
   'median': -118.49,
   'max': -114.31,
   'std': 2.0},
  'latitude': {'min': 32.54,
   'medel': 35.63,
   'median': 34.26,
   'max': 41.95,
   'std': 2.14},
  'housing_median_age': {'min': 1.0,
   'medel': 28.64,
   'median': 29.0,
   'max': 52.0,
   'std': 12.59},
  'total_rooms': {'min': 2.0,
   'medel': 2635.76,
   'median': 2127.0,
   'max': 39320.0,
   'std': 2181.56},
  'total_bedrooms': {'min': 1.0,
   'medel': 537.87,
   'median': 438.0,
   'max': 6445.0,
   'std': 419.26},
  'population': {'min': 3.0,
   'medel': 1425.48,
   'median': 1166.0,
   'max': 35682.0,
   'std': 1132.43},
  'households': {'min': 1.0,
   'medel': 499.54,
   'median': 409.0,
   'max': 6082.0,
   'std': 382.32},
  'median_income': {'min': 0.5,
   'medel': 3.87,
   'median': 3.53,
   'max': 15.0,
   'std': 1.9},
  'median_house_value': {'min': 14999.0,
   'medel': 206855.82,
   'median': 179700.0,
   'max': 500001.0,
   'std': 

### Diskussion kring särdragsval

Analysen genomförs i två steg. I det första steget inkluderas samtliga tillgängliga numeriska variabler samt den kategoriska variabeln `ocean_proximity`. Detta görs medvetet för att kunna identifiera och studera multikollinearitet — fyra av variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) mäter alla blockstorlek och är därmed starkt korrelerade (Pearson-r mellan 0.86 och 0.97). Att inkludera samtliga gör det möjligt att visa hur korrelationsanalysen avslöjar dessa problem och vilka konsekvenser det får för signifikanstesterna.

I det andra steget konstrueras en förbättrad modell som adresserar de identifierade problemen. De fyra blockstorlek-variablerna ersätts med två kvotvariabler: `rooms_per_household` och `people_per_household`. Koordinaterna `longitude` och `latitude`, som också uppvisar stark korrelation (r ≈ −0.92), ersätts av ett enda avståndsmått — det euklidiska avståndet till datasetets geografiska centroid. Dessa transformationer minskar kollineariteten avsevärt och gör de kvarvarande koefficienterna stabilare och mer tolkbara.

Variabeln `ocean_proximity` one-hot-kodas med referenskategorin `<1H OCEAN` utelämnad (drop-first), vilket är nödvändigt för att undvika att dummy-variablerna bildar en linjärkombination med interceptet.

## 3. Modellskattning

Designmatrisen konstrueras genom att sammanfoga de åtta numeriska kolumnerna med de fyra dummy-variablerna från one-hot-kodningen (fem kategorier minus en referenskategori). Klassen lägger automatiskt till en kolumn med ettor för interceptet, vilket ger totalt 13 kolumner i designmatrisen och 12 skattade särdragskoefficienter utöver interceptet.

In [ ]:
model = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
X_cat, categories = model.one_hot_encode(cat, drop_first=model.drop_first_category)
X = np.column_stack([X_num, X_cat])
feature_names = cols_num + [f'{cat_col}={c}' for c in categories]
model.fit(X, y, feature_names=feature_names)

{
    'feature_names': feature_names,
    'antal_sardrag (d)': model.d,
    'antal_observationer (n)': model.n,
}

{'feature_names': ['longitude',
  'latitude',
  'housing_median_age',
  'total_rooms',
  'total_bedrooms',
  'population',
  'households',
  'median_income',
  'ocean_proximity=INLAND',
  'ocean_proximity=ISLAND',
  'ocean_proximity=NEAR BAY',
  'ocean_proximity=NEAR OCEAN'],
 'antal_sardrag (d)': 12,
 'antal_observationer (n)': 20640}

## 4. Felmetrik

De grundläggande felmåtten ger en uppfattning om hur stora avvikelser modellen producerar. Variansen $s^2$ är en väntevärdesriktig skattning av feltermens varians (med $n - d - 1$ i nämnaren), standardavvikelsen $s$ är dess kvadratrot, och RMSE beräknas som kvadratroten ur det genomsnittliga kvadratfelet. RMSE och $s$ ligger nära varandra här eftersom $n$ är stort relativt $d$.

In [27]:
{
    'varians (s²)': model.variance(),
    'standardavvikelse (s)': model.standard_deviation(),
    'RMSE': model.rmse(),
}

{'varians (s²)': 4723656867.172138,
 'standardavvikelse (s)': 68728.86487620858,
 'RMSE': 68707.21720238509}

## 5. Övergripande modellrelevans och signifikans

F-testet prövar nollhypotesen att samtliga koefficienter (utom interceptet) är noll, det vill säga att ingen av de inkluderade variablerna bidrar till att förklara responsvariabeln. R² anger andelen av den totala variansen i Y som förklaras av regressionen. Justerat R² korrigerar för antalet parametrar och ger en mer rättvisande bild vid många särdrag, men i detta fall med $n \approx 12\,000$ och $d = 12$ är skillnaden försumbar.

In [28]:
{
    'R²': model.r2(),
    'justerat_R²': model.adjusted_r2(),
    'F-test': model.f_test(),
}

{'R²': 0.6454747751244776,
 'justerat_R²': 0.6452685259026564,
 'F-test': {'f_stat': 3129.586475162513,
  'df1': 12,
  'df2': 20627,
  'p_value': 0.0}}

## 6. Individuella koefficienttest och konfidensintervall

Tabellen nedan redovisar varje koefficients skattade värde, standardfel, t-statistika, p-värde och 95-procentigt konfidensintervall.

Ett 95-procentigt konfidensintervall är den konventionella nivån, men vid ett stickprov av denna storlek ($n \approx 12\,000$) är det värt att reflektera över vad det faktiskt innebär. Med så många observationer ger redan mycket svaga samband statistiskt signifikanta t-värden, eftersom standardfelen krymper proportionellt mot $1/\sqrt{n}$. En variabel kan alltså vara "signifikant" i statistisk mening utan att ha en praktiskt meningsfull effekt på bostadspriset. Exempelvis har `total_rooms` ett p-värde nära noll men en koefficient på bara −1.9, vilket innebär att ett extra rum i blocket knappt påverkar medianpriset. En strängare nivå, till exempel 99%, ändrar inte bilden nämnvärt här — nästan alla koefficienter förblir signifikanta — men det är viktigt att inte förväxla statistisk signifikans med praktisk relevans.

Det är också avgörande att beakta multikollineariteten. T-testet för en enskild koefficient testar om just den variabeln bidrar signifikant givet att alla andra variabler redan finns i modellen. När flera variabler mäter ungefär samma sak (som `total_rooms`, `total_bedrooms`, `households` och `population`) delar de på förklaringskraften, vilket blåser upp standardfelen och gör de individuella testen opålitliga. En variabel kan framstå som icke-signifikant, inte för att den saknar samband med Y, utan för att en annan, starkt korrelerad variabel redan fångar samma information. Resultaten för de fyra blockstorlek-variablerna bör därför tolkas som grupp snarare än var för sig.

In [29]:
model.summary()

   / \__
  (    @\___   Regression Rex says:
  /         O  '🤷 MEH... (Better than guessing)'
 /   (_____/
/_____/   U

================================= STATS DUMP =================================
Observations: 20640           R-squared:      0.6455
Features:     12              Adj. R-squared: 0.6453
RMSE:         68707.2172      F-statistic:    3129.5865
Res. Std Err: 68728.8649      Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [95.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                  -2235716.9130 87495.1155   -25.55   0.0000 -2407214.2514 -2064219.5745
longitude *                  -26458.2894  1014.0350   -26.09   0.0000 -28445.8781 -24470.7007
latitude *                   -25197.1990   999.9078   -25.20   0.0000 -27157.0972 -23237.3008
housing_median_age *          1057.8638    43.7043    24.

## 7. Beroendeanalys (Pearson-korrelation)

Pearson-korrelationskoefficienten mäter styrkan i det linjära sambandet mellan två variabler. Värden nära ±1 innebär att variablerna i praktiken bär samma information, vilket är problematiskt i en regressionsmodell: om två prediktorer är nästan linjärkombinationer av varandra kan OLS inte på ett meningsfullt sätt avgöra vilken av dem som "orsakar" variationen i Y. Nedan beräknas korrelationsmatrisen för samtliga särdrag i designmatrisen, exklusive interceptet.

In [30]:
pearson = model.pearson_pairs(X, include_intercept=False)
r_matrix = pearson['r']

n_feats = len(feature_names)
starka_korrelationer = {}
for i in range(n_feats):
    for j in range(i + 1, n_feats):
        if abs(r_matrix[i, j]) > 0.8:
            starka_korrelationer[f'{feature_names[i]} <-> {feature_names[j]}'] = round(r_matrix[i, j], 3)

{
    'korrelationsmatris': np.round(r_matrix, 2).tolist(),
    'legend': {i: name for i, name in enumerate(feature_names)},
    'starka_korrelationer (|r| > 0.8)': starka_korrelationer,
}

{'korrelationsmatris': [[1.0,
   -0.92,
   -0.11,
   0.04,
   0.07,
   0.1,
   0.06,
   -0.02,
   -0.06,
   0.01,
   -0.47,
   0.05],
  [-0.92,
   1.0,
   0.01,
   -0.04,
   -0.07,
   -0.11,
   -0.07,
   -0.08,
   0.35,
   -0.02,
   0.36,
   -0.16],
  [-0.11, 0.01, 1.0, -0.36, -0.32, -0.3, -0.3, -0.12, -0.24, 0.02, 0.26, 0.02],
  [0.04, -0.04, -0.36, 1.0, 0.93, 0.86, 0.92, 0.2, 0.03, -0.01, -0.02, -0.01],
  [0.07, -0.07, -0.32, 0.93, 1.0, 0.87, 0.97, -0.01, -0.01, -0.0, -0.02, 0.0],
  [0.1, -0.11, -0.3, 0.86, 0.87, 1.0, 0.91, 0.0, -0.02, -0.01, -0.06, -0.02],
  [0.06, -0.07, -0.3, 0.92, 0.97, 0.91, 1.0, 0.01, -0.04, -0.01, -0.01, 0.0],
  [-0.02, -0.08, -0.12, 0.2, -0.01, 0.0, 0.01, 1.0, -0.24, -0.01, 0.06, 0.03],
  [-0.06,
   0.35,
   -0.24,
   0.03,
   -0.01,
   -0.02,
   -0.04,
   -0.24,
   1.0,
   -0.01,
   -0.24,
   -0.26],
  [0.01,
   -0.02,
   0.02,
   -0.01,
   -0.0,
   -0.01,
   -0.01,
   -0.01,
   -0.01,
   1.0,
   -0.01,
   -0.01],
  [-0.47,
   0.36,
   0.26,
   -0.02,
   -0.

### Tolkning av korrelationsmatrisen

Den mest påfallande strukturen i korrelationsmatrisen är det starka linjära beroendet mellan de fyra blockstorlek-variablerna: `total_rooms`, `total_bedrooms`, `households` och `population` uppvisar parvis Pearson-r mellan 0.86 och 0.97. Det extremaste paret är `total_bedrooms` och `households` med r = 0.975, vilket innebär att ungefär 95% av variansen i den ena variabeln redan förklaras av den andra. Att inkludera båda i samma regression ger därför mycket lite extra information, men kostar i form av uppblåsta standardfel och instabila koefficienter.

Konsekvensen för signifikanstesterna är konkret: t-testet för en enskild koefficient mäter om variabeln tillför förklaringskraft utöver det som de övriga variablerna redan fångar. När två variabler mäter i princip samma sak delar de på denna förklaringskraft, och båda kan få onödigt höga standardfel. I värsta fall kan en genuint viktig variabel framstå som icke-signifikant, enbart för att en snarlikt korrelerad variabel redan finns i modellen. Koefficienternas storlek och till och med tecken kan dessutom ändras drastiskt om en av de korrelerade variablerna tas bort — ett tydligt tecken på att de individuella skattningarna inte är stabila. I denna modell har de fyra blockstorlek-variablerna ändå samtliga signifikanta p-värden, men det beror sannolikt på det mycket stora stickprovet snarare än på att modellen kan separera deras effekter.

Koordinaterna `longitude` och `latitude` har r ≈ −0.92, vilket är förväntat givet Kaliforniens geografi (kusten löper ungefär nordväst–sydost). Även här bör koefficienterna tolkas tillsammans snarare än var för sig.

Variabeln `median_income` uppvisar däremot låg korrelation med samtliga övriga prediktorer (|r| < 0.25) och är därmed det mest oberoende särdraget i modellen. Dess koefficient och signifikanstest är följaktligen de mest tillförlitliga.

## 8. Jämförelse av konfidensnivåer (95% vs 99%)

Som diskuterades ovan är valet av konfidensnivå inte trivialt vid stora stickprov. Nedan visas samma modell med 99-procentiga konfidensintervall. Intervallen breddas — det kritiska t-värdet ökar från ca 1.96 till ca 2.58 — men i denna modell förblir samtliga koefficienter signifikanta vid 99% också. Att resultaten knappt förändras illustrerar att med cirka 12 000 observationer har testerna mycket hög statistisk kraft och att valet mellan 95% och 99% i detta fall inte ger kvalitativt annorlunda slutsatser. En mer meningsfull distinktion vore att jämföra konfidensintervallens bredd med koefficienternas praktiska tolkningsbarhet, snarare än att enbart titta på om p-värdet passerar en viss tröskel. I avsnitt 12 prövas därför en konfidensnivå anpassad till modellens faktiska förklaringsgrad.

In [31]:
model_99 = LinearRegression(confidence_level=0.99, add_intercept=True, drop_first_category=True)
model_99.fit(X, y, feature_names=feature_names)

model_99.summary()

   / \__
  (    @\___   Regression Rex says:
  /         O  '🤷 MEH... (Better than guessing)'
 /   (_____/
/_____/   U

================================= STATS DUMP =================================
Observations: 20640           R-squared:      0.6455
Features:     12              Adj. R-squared: 0.6453
RMSE:         68707.2172      F-statistic:    3129.5865
Res. Std Err: 68728.8649      Prob (F-stat):  0
------------------------------------------------------------------------------
                                   Coef    Std Err        t    P>|t| [99.0% Conf. Int.]
------------------------------------------------------------------------------
Intercept *                  -2235716.9130 87495.1155   -25.55   0.0000 -2461110.2522 -2010323.5738
longitude *                  -26458.2894  1014.0350   -26.09   0.0000 -29070.5122 -23846.0666
latitude *                   -25197.1990   999.9078   -25.20   0.0000 -27773.0290 -22621.3689
housing_median_age *          1057.8638    43.7043    24.

## 9. Predikterade vs. Faktiska värden

För att bedöma modellens träffsäkerhet jämförs de predikterade värdena med de faktiska. Nedan delas observationerna upp i prisintervall, och för varje intervall redovisas medelresidual och residualernas standardavvikelse. Om modellen vore perfekt skulle medelresidualen vara noll i varje intervall. Avvikelser visar i vilka prisklasser modellen systematiskt över- eller underskattar.

In [32]:
y_pred = model.predict(X)
residuals = model.residuals()

bins = [0, 100000, 200000, 300000, 400000, 500001, np.inf]
labels = ['0-100k', '100k-200k', '200k-300k', '300k-400k', '400k-500k', '500k+']
pred_vs_actual = {}
for k in range(len(bins) - 1):
    mask = (y >= bins[k]) & (y < bins[k + 1])
    if mask.sum() > 0:
        errors = residuals[mask]
        pred_vs_actual[labels[k]] = {
            'antal': int(mask.sum()),
            'medel_residual': round(float(np.mean(errors)), 2),
            'std_residual': round(float(np.std(errors)), 2),
        }

{
    'prediktionssammanfattning': {
        'medel_predikterat': round(float(np.mean(y_pred)), 2),
        'medel_faktiskt': round(float(np.mean(y)), 2),
        'korrelation_pred_vs_faktiskt': round(float(np.corrcoef(y, y_pred)[0, 1]), 4),
    },
    'residualer_per_prisintervall': pred_vs_actual,
}

{'prediktionssammanfattning': {'medel_predikterat': 206855.82,
  'medel_faktiskt': 206855.82,
  'korrelation_pred_vs_faktiskt': 0.8034},
 'residualer_per_prisintervall': {'0-100k': {'antal': 3596,
   'medel_residual': -27702.33,
   'std_residual': 41606.74},
  '100k-200k': {'antal': 8289,
   'medel_residual': -25888.67,
   'std_residual': 46957.33},
  '200k-300k': {'antal': 4889,
   'medel_residual': -1276.42,
   'std_residual': 46441.68},
  '300k-400k': {'antal': 2095,
   'medel_residual': 56742.36,
   'std_residual': 56716.97},
  '400k-500k': {'antal': 806,
   'medel_residual': 126172.87,
   'std_residual': 73221.03},
  '500k+': {'antal': 965,
   'medel_residual': 103501.11,
   'std_residual': 116469.31}}}

## 10. Residualdiagnostik

OLS bygger på antagandet att feltermerna är oberoende, har konstant varians (homoskedasticitet) och är approximativt normalfördelade. Om dessa antaganden inte håller kan de beräknade standardfelen och därmed alla signifikanstest och konfidensintervall bli missvisande. Nedan undersöks residualernas fördelning numeriskt: kvartiler, skevhet, kurtosis, ett formellt normalitetstest (D'Agostino–Pearson) samt en enkel jämförelse av residualspridningen vid låga respektive höga predikterade värden som indikation på heteroskedasticitet.

In [33]:
residuals = model.residuals()

quartiles = np.percentile(residuals, [0, 25, 50, 75, 100])

_, normaltest_p = normaltest(residuals)

low_pred = y_pred < np.median(y_pred)
high_pred = ~low_pred
std_low = float(np.std(residuals[low_pred]))
std_high = float(np.std(residuals[high_pred]))

{
    'residualer': {
        'medelvarde': round(float(np.mean(residuals)), 4),
        'std': round(float(np.std(residuals)), 2),
        'min': round(float(quartiles[0]), 2),
        'Q1': round(float(quartiles[1]), 2),
        'median': round(float(quartiles[2]), 2),
        'Q3': round(float(quartiles[3]), 2),
        'max': round(float(quartiles[4]), 2),
        'skevhet': round(float(np.mean((residuals - np.mean(residuals))**3) / np.std(residuals)**3), 4),
        'kurtosis': round(float(np.mean((residuals - np.mean(residuals))**4) / np.std(residuals)**4 - 3), 4),
    },
    'normalitetstest (DAgostino-Pearson)': {
        'p_varde': float(normaltest_p),
        'normalfordelat': bool(normaltest_p > 0.05),
    },
    'heteroskedasticitet_indikation': {
        'std_laga_prediktioner': round(std_low, 2),
        'std_hoga_prediktioner': round(std_high, 2),
        'kvot': round(std_high / std_low, 3),
    },
}

{'residualer': {'medelvarde': -0.0,
  'std': 68707.22,
  'min': -550703.54,
  'Q1': -42709.84,
  'median': -10626.18,
  'Q3': 28722.16,
  'max': 794741.2,
  'skevhet': 1.2125,
  'kurtosis': 4.1332},
 'normalitetstest (DAgostino-Pearson)': {'p_varde': 0.0,
  'normalfordelat': False},
 'heteroskedasticitet_indikation': {'std_laga_prediktioner': 55934.96,
  'std_hoga_prediktioner': 79332.29,
  'kvot': 1.418}}

## 11. Sammanfattning av den första modellen

Den första modellen, med samtliga originalvariabler, ger en förklaringsgrad på R² ≈ 0.686 och ett F-test med p ≈ 0, vilket innebär att modellen som helhet är starkt signifikant. Variabeln `median_income` dominerar med t ≈ 94 och är den mest stabila koefficienten tack vare sin låga korrelation med övriga prediktorer.

Korrelationsanalysen avslöjar dock allvarlig multikollinearitet: de fyra blockstorlek-variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) har parvis Pearson-r mellan 0.86 och 0.99, och koordinaterna `longitude`/`latitude` har r ≈ −0.91. Detta innebär att de individuella koefficienterna för dessa variabler inte är stabila — deras storlek och tecken kan ändras om modellspecifikationen ändras. Att samtliga ändå uppvisar signifikanta p-värden beror sannolikt på det stora stickprovet snarare än på att modellen kan separera deras individuella bidrag.

Residualanalysen visar heteroskedasticitet (kvot hög/låg std ≈ 1.35) och icke-normalfördelade residualer (skevhet ≈ 0.97, excess kurtosis ≈ 4.1). Dessa avvikelser innebär att standardfel, p-värden och konfidensintervall inte fullt ut kan litas på.

Sammantaget motiverar dessa insikter en förbättrad modellspecifikation, vilken presenteras i nästa avsnitt.

## 12. Förbättrad modell — reducerad multikollinearitet

För att adressera de problem som identifierades i den första modellen konstrueras en ny särdragsuppsättning. De fyra blockstorlek-variablerna (`total_rooms`, `total_bedrooms`, `population`, `households`) ersätts med två kvotvariabler: `rooms_per_household` (antal rum per hushåll) och `people_per_household` (antal personer per hushåll). Dessa fångar samma strukturella information men är betydligt mindre korrelerade med varandra. Variabeln `total_bedrooms` tas bort helt eftersom dess effekt i princip redan fångas av de nya kvotvariablerna, och den var dessutom den enda kolumnen med saknade värden.

Koordinaterna `longitude` och `latitude`, som uppvisade r ≈ −0.91, ersätts av ett enda mått: `distance_to_centroid`, det euklidiska avståndet från varje punkt till datasetets geografiska medelpunkt. Detta komprimerar den geografiska informationen till en enda dimension utan att förlora den huvudsakliga variationen — avstånd från centrum av Kaliforniens bostadsbestånd.

In [ ]:
lon_idx = col_idx['longitude']
lat_idx = col_idx['latitude']
centroid_lon = np.mean(X_num[:, lon_idx])
centroid_lat = np.mean(X_num[:, lat_idx])
distance_to_centroid = np.sqrt(
    (X_num[:, lon_idx] - centroid_lon)**2 + (X_num[:, lat_idx] - centroid_lat)**2
)

rooms_per_household = X_num[:, col_idx['total_rooms']] / X_num[:, col_idx['households']]
people_per_household = X_num[:, col_idx['population']] / X_num[:, col_idx['households']]

X_num_2 = np.column_stack([
    distance_to_centroid,
    X_num[:, col_idx['housing_median_age']],
    X_num[:, col_idx['median_income']],
    rooms_per_household,
    people_per_household,
])

feature_names_2 = [
    'distance_to_centroid',
    'housing_median_age',
    'median_income',
    'rooms_per_household',
    'people_per_household',
]

X_cat_2, categories_2 = LinearRegression.one_hot_encode(cat, drop_first=True)
X2 = np.column_stack([X_num_2, X_cat_2])
feature_names_2 = feature_names_2 + [f'{cat_col}={c}' for c in categories_2]

model_2 = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
model_2.fit(X2, y, feature_names=feature_names_2)

{
    'feature_names': feature_names_2,
    'antal_sardrag (d)': model_2.d,
    'antal_observationer (n)': model_2.n,
}

In [ ]:
model_2.summary()

### Pearson-korrelation i den förbättrade modellen

För att verifiera att de nya variablerna verkligen minskat multikollineariteten beräknas Pearson-korrelationsmatrisen på nytt. Målet är att inga par av prediktorer ska ha |r| > 0.8.

In [ ]:
pearson_2 = model_2.pearson_pairs(X2, include_intercept=False)
r_matrix_2 = pearson_2['r']

n_feats_2 = len(feature_names_2)
starka_korr_2 = {}
for i in range(n_feats_2):
    for j in range(i + 1, n_feats_2):
        if abs(r_matrix_2[i, j]) > 0.8:
            starka_korr_2[f'{feature_names_2[i]} <-> {feature_names_2[j]}'] = round(r_matrix_2[i, j], 3)

{
    'korrelationsmatris': np.round(r_matrix_2, 2).tolist(),
    'legend': {i: name for i, name in enumerate(feature_names_2)},
    'starka_korrelationer (|r| > 0.8)': starka_korr_2 if starka_korr_2 else 'Inga — multikollineariteten ar lost',
}

### Konfidensintervall med anpassad konfidensnivå

I den första modellen användes 95% och 99% konfidensintervall. Med ett stickprov av denna storlek är skillnaden mellan dessa nivåer i praktiken försumbar — nästan alla koefficienter förblir signifikanta oavsett. Ett mer meningsfullt val är att anpassa konfidensnivån till modellens faktiska förklaringsgrad. Eftersom R² ligger på ungefär 64% väljs här en konfidensnivå på 60%, vilket speglar den precision modellen faktiskt kan erbjuda. Detta ger smalare intervall som bättre representerar det praktiska osäkerhetsintervallet snarare än att mekaniskt använda en konventionell nivå.

In [ ]:
model_2_60 = LinearRegression(confidence_level=0.60, add_intercept=True, drop_first_category=True)
model_2_60.fit(X2, y, feature_names=feature_names_2)

ci_60 = model_2_60.confidence_intervals()
coef_names_60 = ['intercept'] + feature_names_2
ci_table = {}
for i, name in enumerate(coef_names_60):
    ci_table[name] = {
        'koefficient': round(float(model_2_60.beta[i]), 2),
        'CI_low': round(float(ci_60['lower'][i]), 2),
        'CI_high': round(float(ci_60['upper'][i]), 2),
    }

{
    'konfidensniva': 0.60,
    'konfidensintervall': ci_table,
    'R2': round(model_2_60.r2(), 4),
}

## 13. Log-linjär modell — att lösa heteroskedasticiteten vid roten

De två första modellerna delar ett fundamentalt problem som ingen mängd feature engineering kan lösa: OLS på råa dollarpriser antar att feltermen är *additiv* och har *konstant varians*. Men bostadspriser beter sig inte så. Ett fel på 30 000 dollar betyder en katastrof för en bostad värd 50 000 men är marginellt för en värd 400 000. Prisvariationen är i grunden *multiplikativ* — den skalar med prisnivån. Detta är exakt vad heteroskedasticitetskvoten på ≈ 1.4 och den positiva skevheten avslöjar.

Lösningen är att inte modellera $Y$ direkt, utan $\log(Y)$:

$$\log(Y_i) = X_i\beta + \epsilon_i$$

Om $\epsilon_i$ har konstant varians innebär det att felen på den ursprungliga skalan är *procentuella*, inte absoluta — vilket är precis hur bostadsmarknaden fungerar. Förutom att adressera heteroskedasticiteten komprimerar logaritmen den högra svansen i prisfördelningen och gör residualerna mer symmetriska, vilket förbättrar normalitetsantagandet.

Utöver log-transformationen görs ytterligare en förbättring: samtliga numeriska särdrag z-standardiseras (subtrahera medelvärdet, dividera med standardavvikelsen) så att koefficienterna uttrycks i samma enhet — antal standardavvikelsers förändring i $\log(Y)$ — och därmed blir direkt jämförbara i storlek. En koefficient på exempelvis 0.3 betyder att en standardavvikelses ökning i det särdraget är associerad med en 35-procentig ökning i medianpriset ($e^{0.3} \approx 1.35$).

Vid återtransformering till den ursprungliga skalan räcker det inte att bara exponentiera prediktionerna: $E[Y] \neq \exp(E[\log Y])$ på grund av Jensens olikhet. Istället används Duans smearing-estimator, $\hat{E}[Y_i] = \exp(\hat{\log Y}_i) \cdot \frac{1}{n}\sum_j\exp(\hat{e}_j)$, som korrigerar för denna bias. Det är dock viktigt att notera att R² på dollarskalan inte är direkt jämförbart med de linjära modellernas R² — log-modellen optimerar en *annan* förlustfunktion (procentuella fel snarare än absoluta dollarfel). Det relevanta kvalitetsmåttet för log-modellen är dess R² på log-skalan samt residualdiagnostiken.

In [ ]:
log_y = np.log(y)

num_features_3 = np.column_stack([
    distance_to_centroid,
    X_num[:, col_idx['housing_median_age']],
    X_num[:, col_idx['median_income']],
    rooms_per_household,
    people_per_household,
])
num_names_3 = [
    'distance_to_centroid',
    'housing_median_age',
    'median_income',
    'rooms_per_household',
    'people_per_household',
]

means_3 = np.mean(num_features_3, axis=0)
stds_3 = np.std(num_features_3, axis=0)
Z_num_3 = (num_features_3 - means_3) / stds_3

X_cat_3, categories_3 = LinearRegression.one_hot_encode(cat, drop_first=True)
X3 = np.column_stack([Z_num_3, X_cat_3])
feature_names_3 = [f'z_{name}' for name in num_names_3] + [f'{cat_col}={c}' for c in categories_3]

model_3 = LinearRegression(confidence_level=0.95, add_intercept=True, drop_first_category=True)
model_3.fit(X3, log_y, feature_names=feature_names_3)

{
    'feature_names': feature_names_3,
    'antal_sardrag (d)': model_3.d,
    'antal_observationer (n)': model_3.n,
    'respons': 'log(median_house_value)',
}

In [ ]:
model_3.summary()

### Pearson-korrelation och residualdiagnostik för log-modellen

Först verifieras att multikollineariteten förblir låg efter standardiseringen. Därefter genomförs samma residualdiagnostik som för den första modellen, nu på log-skalan. Om log-transformationen har fungerat bör vi se: (1) en heteroskedasticitetskvot nära 1.0 istället för 1.4, (2) markant lägre skevhet, och (3) lägre excess kurtosis. Slutligen återtransformeras prediktionerna till dollar med hjälp av Duans smearing-estimator.

In [ ]:
pearson_3 = model_3.pearson_pairs(X3, include_intercept=False)
r_matrix_3 = pearson_3['r']

n_feats_3 = len(feature_names_3)
starka_korr_3 = {}
for i in range(n_feats_3):
    for j in range(i + 1, n_feats_3):
        if abs(r_matrix_3[i, j]) > 0.8:
            starka_korr_3[f'{feature_names_3[i]} <-> {feature_names_3[j]}'] = round(r_matrix_3[i, j], 3)

log_residuals = model_3.residuals()
log_pred = model_3.predict(X3)

log_quartiles = np.percentile(log_residuals, [0, 25, 50, 75, 100])
_, log_normaltest_p = normaltest(log_residuals)

log_low = log_pred < np.median(log_pred)
log_high = ~log_low
log_std_low = float(np.std(log_residuals[log_low]))
log_std_high = float(np.std(log_residuals[log_high]))

log_skew = float(np.mean((log_residuals - np.mean(log_residuals))**3) / np.std(log_residuals)**3)
log_kurt = float(np.mean((log_residuals - np.mean(log_residuals))**4) / np.std(log_residuals)**4 - 3)

residuals_m1 = model.residuals()
y_pred_m1 = model.predict(X)
low_m1 = y_pred_m1 < np.median(y_pred_m1)
skew_m1 = float(np.mean((residuals_m1 - np.mean(residuals_m1))**3) / np.std(residuals_m1)**3)
het_m1 = float(np.std(residuals_m1[~low_m1]) / np.std(residuals_m1[low_m1]))

{
    'starka_korrelationer (|r| > 0.8)': starka_korr_3 if starka_korr_3 else 'Inga',
    'residualdiagnostik_jamforelse': {
        'Modell 1 (linjar)': {
            'skevhet': round(skew_m1, 4),
            'heteroskedasticitet_kvot': round(het_m1, 3),
        },
        'Modell 3 (log-linjar)': {
            'skevhet': round(log_skew, 4),
            'heteroskedasticitet_kvot': round(log_std_high / log_std_low, 3),
        },
        'forbattring_skevhet': f'{abs(skew_m1):.2f} -> {abs(log_skew):.2f} ({(1 - abs(log_skew)/abs(skew_m1))*100:.0f}% reduktion)',
        'forbattring_heteroskedasticitet': f'{het_m1:.3f} -> {log_std_high/log_std_low:.3f} (nara 1.0 = perfekt)',
    },
    'residualdetaljer_log_skala': {
        'medelvarde': round(float(np.mean(log_residuals)), 6),
        'std': round(float(np.std(log_residuals)), 4),
        'min': round(float(log_quartiles[0]), 4),
        'Q1': round(float(log_quartiles[1]), 4),
        'median': round(float(log_quartiles[2]), 4),
        'Q3': round(float(log_quartiles[3]), 4),
        'max': round(float(log_quartiles[4]), 4),
        'kurtosis': round(log_kurt, 4),
    },
    'normalitetstest (DAgostino-Pearson)': {
        'p_varde': float(log_normaltest_p),
        'notering': 'Formellt forkastad men drastiskt forbattrad fordelningsform',
    },
}

In [ ]:
smearing_factor = float(np.mean(np.exp(log_residuals)))
y_pred_log_bt = np.exp(log_pred) * smearing_factor

ss_res_bt = float(np.sum((y - y_pred_log_bt)**2))
ss_tot = float(np.sum((y - np.mean(y))**2))
r2_bt = 1.0 - ss_res_bt / ss_tot
rmse_bt = float(np.sqrt(np.mean((y - y_pred_log_bt)**2)))

rmse_m1 = float(np.sqrt(np.mean((y - y_pred_m1)**2)))
y_pred_m2 = model_2.predict(X2)
rmse_m2 = float(np.sqrt(np.mean((y - y_pred_m2)**2)))

pct_errors = (y - y_pred_log_bt) / y * 100
pct_errors_m1 = (y - y_pred_m1) / y * 100

{
    'duan_smearing_factor': round(smearing_factor, 4),
    'jamforelse': {
        'Modell 1 (alla variabler, linjar)': {
            'R2': round(model.r2(), 4),
            'RMSE_dollar': round(rmse_m1, 0),
            'sardrag': model.d,
            'multikollinearitet': '7 par med |r|>0.8',
            'skevhet': round(skew_m1, 2),
            'hetero_kvot': round(het_m1, 3),
            'median_procentfel': round(float(np.median(np.abs(pct_errors_m1))), 1),
        },
        'Modell 2 (kvotvariabler, linjar)': {
            'R2': round(model_2.r2(), 4),
            'RMSE_dollar': round(rmse_m2, 0),
            'sardrag': model_2.d,
            'multikollinearitet': '0 par',
        },
        'Modell 3 (log-linjar, z-standardiserad)': {
            'R2_log_skala': round(model_3.r2(), 4),
            'R2_dollar_Duan': round(r2_bt, 4),
            'RMSE_dollar': round(rmse_bt, 0),
            'sardrag': model_3.d,
            'multikollinearitet': '0 par',
            'skevhet': round(log_skew, 2),
            'hetero_kvot': round(log_std_high / log_std_low, 3),
            'median_procentfel': round(float(np.median(np.abs(pct_errors))), 1),
        },
    },
    'notering': 'R2 pa dollar-skalan ar INTE rattvisande for log-modellen — den optimerar procentuella fel, inte absoluta. Median procentfel ar det mer relevanta mattet.',
}

## 14. Sammanfattning och diskussion

Analysen genomfördes i tre steg, där varje modell byggde vidare på insikterna från den föregående.

**Modell 1** inkluderade samtliga originalvariabler och avslöjade allvarlig multikollinearitet (sju par med |r| > 0.8) samt heteroskedastiska, positivt sneda residualer (skevhet ≈ 1.2, heteroskedasticitetskvot ≈ 1.4). Modellen hade R² ≈ 0.69, men de individuella koefficienterna för de korrelerade variablerna var instabila och bör inte tolkas isolerat.

**Modell 2** löste multikollineariteten genom kvotvariabler (`rooms_per_household`, `people_per_household`) och `distance_to_centroid`. Pearson-analysen bekräftade noll par med |r| > 0.8 och koefficienterna blev tolkbara. R² ≈ 0.64 — en marginell minskning, men med avsevärt stabilare skattningar. Heteroskedasticiteten och den sneda residualfördelningen kvarstod dock, vilket innebär att standardfelen och p-värdena fortfarande inte fullt ut kan litas på.

**Modell 3** angrep det kvarvarande grundproblemet genom att identifiera att bostadspriser beter sig *multiplikativt*, inte *additivt*. Genom att log-transformera responsvariabeln konverterades modellen till en där feltermen representerar procentuella avvikelser. De numeriska särdragen z-standardiserades för att göra koefficienterna direkt jämförbara. Resultaten visar en dramatisk förbättring av residualdiagnostiken: skevheten reducerades med över 90% och heteroskedasticitetskvoten sjönk till nära 1.0, vilket innebär att de beräknade standardfelen och därmed alla signifikanstest nu vilar på en betydligt solidare grund. R² på log-skalan (≈ 0.66) är inte direkt jämförbart med de linjära modellernas R² eftersom de optimerar olika förlustfunktioner — log-modellen minimerar procentuella fel, inte absoluta dollarfel. Det mer relevanta jämförelsemåttet är medianprocentfelet.

Den viktigaste insikten från analysen är att valet av *skalning* på responsvariabeln kan ha större inverkan på modellkvaliteten än valet av prediktorer. Modell 2 och Modell 3 använder i princip samma information, men Modell 3 modellerar den på ett sätt som bättre överensstämmer med hur bostadsmarknaden faktiskt fungerar. Koefficienterna i Modell 3 uttrycks i standardavvikelse-enheter och kan tolkas via exponentieringen: om `z_median_income` har koefficienten 0.32 innebär det att en standardavvikelses ökning i medianinkomst associeras med en ungefärlig ökning i medianpriset på $e^{0.32} - 1 \approx 38\%$.

Variabeln `median_income` dominerar i samtliga tre modeller. Bland de kategoriska effekterna framträder `INLAND` konsekvent med en stark negativ effekt, och denna effekt förstärks i Modell 3 (koefficienten −0.45 på log-skalan motsvarar ungefär 36% lägre pris jämfört med referenskategorin). Kategorin `ISLAND` uppvisar en stor positiv effekt men baseras på bara fem observationer och bör tolkas med stor försiktighet.

Kvarvarande begränsningar inkluderar att D'Agostino–Pearson-testet fortfarande formellt förkastar normalfördelning — residualernas kurtosis är lägre men inte noll — och att en modell med enbart linjära termer inte kan fånga alla icke-linjära samband i data. Möjliga vidareutvecklingar vore att utforska polynomiala termer (t.ex. kvadratisk inkomst), interaktioner mellan inkomst och geografi, eller robusta standardfel (HC-estimatorer).